In [1]:
import findspark
findspark.init()
print("This is  a test program")

This is  a test program


In [2]:
from pyspark import SparkConf,SparkContext
from pyspark.sql import SQLContext
from pyspark.sql.session import SparkSession
from pyspark.sql import Row
from pyspark.sql import HiveContext
import random
conf=SparkConf().setMaster("local").setAppName("SparkContext_HIVE2")
sc=SparkContext.getOrCreate(conf=conf)

print("Spark Context")
print(sc)


sql_context=SQLContext(sc)
hive_context=HiveContext(sc)
print("Hive Context")
print(hive_context)

print("SQL Context")
print(sql_context)

spark=SparkSession(sc)

print("Spark")
print(spark)


RecSales= Row('ordernumber','quantityordered','priceeach','orderlinenumber','sales','orderdate','status','qtr_id','month_id','year_id','productline','msrp','productcode','customername','phone','addressline1','addressline2','city','state','postalcode','country','territory','contactlastname','contactfirstname','dealsize')
dataSales=sc.textFile("/opt/sparktest/sales_data_sample.csv")
header=dataSales.first()
dataSales=dataSales.filter(lambda line: line!=header)
recSales=dataSales.map(lambda l: l.split(","))
dataSales=recSales.map(lambda l: RecSales(l[0],l[1],l[2],l[3],l[4],l[5],l[6],l[7],l[8],l[9],l[10],l[11],l[12],l[13],l[14],l[15],l[16],l[17],l[18],l[19],l[20],l[21],l[22],l[23],l[24]))
dfrecSales=hive_context.createDataFrame(dataSales)
dfrecSales.registerTempTable("sales")

res1=hive_context.sql("select count(*) from sales").collect()
print(res1)
res2=hive_context.sql("select * from sales").head()
print(res2)
res3=hive_context.sql("select ordernumber,priceeach from sales").head(2)
print(res3)

dfterritory=hive_context.sql("select territory,sum(priceeach) total from sales group by territory")
dfterritory.registerTempTable("sumterr")
dfterritory.show()

print("Create Hive Table and Write the Results")
hive_context.sql("create table if not exists territory_sum (territory String, total INT)")
hive_context.sql("insert into territory_sum select * from sumterr")

print("Verify if data is written to Hive")
print("-----------------------------------")

print("Query1: select * from territory_sum")
hive_context.sql("select * from territory_sum").show()

print("Query2: select count(*) from territory_sum")
hive_context.sql("select count(*) from territory_sum").show()

print("Query3: select count(*) from sumterr")
hive_context.sql("select count(*) from sumterr").show()

sc.stop()

Spark Context
<SparkContext master=local appName=SparkContext_HIVE2>
Hive Context
SQL Context
Spark
[Row(count(1)=2823)]
Row(ordernumber='10107', quantityordered='30', priceeach='95.7', orderlinenumber='2', sales='2871', orderdate='2/24/2003 0:00', status='Shipped', qtr_id='1', month_id='2', year_id='2003', productline='Motorcycles', msrp='95', productcode='S10_1678', customername='Land of Toys Inc.', phone='2125557818', addressline1='897 Long Airport Avenue', addressline2='', city='NYC', state='NY', postalcode='10022', country='USA', territory='NA', contactlastname='Yu', contactfirstname='Kwai', dealsize='Small')
[Row(ordernumber='10107', priceeach='95.7'), Row(ordernumber='10121', priceeach='81.35')]
+-----------+------------------+
|  territory|             total|
+-----------+------------------+
|     Sweden|           1720.14|
|    Germany|            5184.3|
|     France|13744.239999999994|
|         NA|  84928.1399999999|
|      41101|           1155.06|
|    Belgium| 727.819999